<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/missions_heasarc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HEASARC** (Ultraviolet, X-Ray, Gamma Ray)

![sciences](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_2000.png)

In [ ]:
!pip install astropy astroquery requests reproject -q
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astropy import coordinates
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from reproject import reproject_interp
import matplotlib.pyplot as plt

In [ ]:
# Some tasks like multi-wavelength astronomy require high memory to load data
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Runtime: {:.1f} gigabytes of available RAM (>20 GB ideal)\n'.format(ram_gb))

Runtime: 54.8 gigabytes of available RAM (>20 GB ideal)



#### **Define astronomical object and get skycoordinates**

In [ ]:
# Define object
object_name = 'M51'

*e.g. M31 (Andromeda galaxy), M104 (Sombrero galaxy), M51 (Whirlpool galaxy), M42 (Orion nebula), NGC 3372 (Carina nebula)*

In [ ]:
# Get and set skycoordinates of object
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

skyobject = Simbad.query_object(object_name)
ra_hms = skyobject['RA'][0]
dec_dms = skyobject['DEC'][0]
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
print(skyobject)
print("\nCoordinates:")
print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

MAIN_ID      RA          DEC      RA_PREC ... COO_WAVELENGTH     COO_BIBCODE     SCRIPT_NUMBER_ID
          "h:m:s"      "d:m:s"            ...                                                    
------- ------------ ------------ ------- ... -------------- ------------------- ----------------
  M  51 13 29 52.698 +47 11 42.93       7 ...              I 2006AJ....131.1163S                1

Coordinates:
RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Convert from Right Ascension (RA) and Declination (DEC) to decimal degree
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
ra_deg = coord.ra.deg # round(coord.ra.deg, 2)
dec_deg = coord.dec.deg # round(coord.dec.deg, 2)

print("RA in degrees:", ra_deg)
print("DEC in degrees:", dec_deg)

RA in degrees: 202.46957499999996
DEC in degrees: 47.19525833333333


In [ ]:
# Convert from decimal degree to Right Ascension (RA) and Declination (DEC)
coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame='icrs')

# RA: hours, minutes, seconds. DEC: degrees, arcminutes, arcseconds
ra_hms = coord.ra.to_string(unit='hour', sep=' ', precision=3, pad=True)
dec_dms = coord.dec.to_string(unit='deg', sep=' ', precision=2, alwayssign=True, pad=True)

print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Reverse check to see to which object the coordinates belong
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M  51


frame='icrs' refers to [International Celestial Ref. System](https://aa.usno.navy.mil/faq/ICRS_doc)
*  Origin is barycenter of Solar System. This means ICRS origin is not fixed to Earth but moves with Solar System's center of mass.
* Axes are defined by the positions of extragalactic sources, primarily quasars observed using Very Long Baseline Interferometry (VLBI). The primary plane is as close as possible to the mean equator at J2000.0 (January 1, 2000, 12:00 TT).  The origin of right ascension is close to the dynamical equinox of J2000.0.
* Accuracy: The ICRF3 defines the ICRS axes with an accuracy of about 20 microarcseconds.
* Realization: The International Celestial Reference Frame (ICRF) is a catalog of precise positions of extragalactic sources that provides a practical realization of the ICRS. The latest version, ICRF3, includes the positions of over 4000 extragalactic sources, mostly quasars. The ICRF is maintained by the International Earth Rotation and Reference Systems Service (IERS).
* Relationship to other systems: ICRS is closely aligned with the J2000 equatorial coordinate system, but it's more accurate and stable. Transformations exist to convert coordinates between ICRS and other celestial reference systems

In [ ]:
# Set coordinates manually and verify correct object
coord = SkyCoord('189.9976d -11.623054d', frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M 104


#### **HEASARC (Ultraviolet, X-Ray, Gamma Ray)**

HEASARC (High Energy Astrophysics Science Archive Research Center): data from missions observing the high-energy universe, like X-rays and gamma rays. Like black holes, supernovas, and active galaxies
* Missions: Chandra, XMM-Newton, Swift, Fermi, NuSTAR
* Website: heasarc.gsfc.nasa.gov

https://astroquery.readthedocs.io/en/latest/heasarc/heasarc.html

In [ ]:
# Query all available missions from library
from astroquery.heasarc import Heasarc
heasarc = Heasarc()
missions = heasarc.query_mission_list()
print(missions)

 Mission     Table                                  Table Description                               
---------- ---------- ------------------------------------------------------------------------------
     AGILE   agilecat                       First AGILE Catalog of High-Confidence Gamma-Ray Sources
     AGILE agileupvar              Updated AGILE Catalog of Bright Gamma-Ray Sources and Variability
     AKARI   akaribsc                      AKARI/FIS All-Sky Survey Bright Source Catalog, Version 1
     AKARI   akaripsc                       AKARI/IRC All-Sky Survey Point Source Catalog, Version 1
       ANS ansuvpscat                            ANS Ultraviolet Photometry Catalog of Point Sources
   ARIEL V    ariel3a                                                        3rd Ariel-V SSI Catalog
   ARIEL V     ariel5                                                        Ariel V All-Sky Monitor
      ASCA ascaegclus                           ASCA Elliptical Galaxies and Galaxy Cluster

Main X-ray missions:
- `chanmaster`: Chandra Master Catalog
- `csc2master`: Chandra Source Catalog 2.0
- `xmmmaster`: XMM-Newton Master Catalog
- `xmmssc`: XMM-Newton Serendipitous Source Catalog
- `swiftmastr`: Swift Master Catalog
- `swiftxrtspec`: Swift-XRT Spectral Database
- `rass2rxs`: ROSAT All-Sky Survey Catalogs
- `exomaster`: EXOSAT Master Catalog
- `numaster`: NuSTAR Master Catalog

For other wavelengths/multi-wavelength:
- `fermimastr`: Fermi Master Catalog (gamma-ray)
- `intmaster`: INTEGRAL Master Catalog (gamma-ray)
- `suzamaster`: Suzaku Master Catalog
- `hstadvize`: Hubble Space Telescope Pointed Observations (optical)
- `jwstmaster`: James Webb Space Telescope Master Catalog (infrared)

In [ ]:
# Select mission
mission='chanmaster' # try also 'swiftmastr'

In [ ]:
# Query all observations from selected mission (PI - Principal Investigator. Lead scientist for observation)
mission_data = heasarc.query_object('*', mission)
print(mission_data)

OBSID        STATUS        ...                    PI                    PUBLIC_DATE
                           ...                                              mjd    
----- -------------------- ... ---------------------------------------- -----------
 6616 archived             ... Miller                                         54159
 7587 archived             ... Canizares                                      54502
28092 untriggered          ... Canizares                                          0
29852 untriggered          ... Canizares                                          0
28354 unobserved           ... Kargaltsev                                         0
30481 untriggered          ... Canizares                                          0
28775 untriggered          ... Canizares                                          0
13814 archived             ... Kuntz                                          56576
13842 archived             ... Baganoff                                     

In [ ]:
# Query a specific object in mission
object_data = heasarc.query_object(object_name, mission)
print(object_data)

OBSID        STATUS        ... PUBLIC_DATE SEARCH_OFFSET_
                           ...     mjd                   
----- -------------------- ... ----------- --------------
 9533 archived             ...       55169 0.000 (M104)\n
 9532 archived             ...       54952 0.500 (M104)\n
 1586 archived             ...       52431 0.135 (M104)\n
  407 archived             ...       51920 0.176 (M104)\n


In [ ]:
# List all possible columns
print(object_data.info)

Available columns:
<Table length=4>
     name       dtype  unit  format
-------------- ------- ---- -------
         OBSID   int32             
        STATUS bytes20             
          NAME bytes50             
            RA float64  deg {:9.5f}
           DEC float64  deg {:9.5f}
          TIME bytes16  mjd        
      DETECTOR  bytes6             
       GRATING  bytes4             
      EXPOSURE float64    s {:5.0f}
          TYPE  bytes3             
            PI bytes40             
   PUBLIC_DATE   int32  mjd        
SEARCH_OFFSET_ bytes13             



In [ ]:
# Reduce to specific Query specific columns for an object
specific_data = heasarc.query_object(object_name, mission, fields='OBSID,ra,dec,detector')
print(specific_data)

OBSID     RA       DEC    DETECTOR SEARCH_OFFSET_
         deg       deg                           
----- --------- --------- -------- --------------
 9533 189.99763 -11.62306   ACIS-I 0.000 (M104)\n
 1586 189.99583 -11.62444   ACIS-S 0.135 (M104)\n
  407 189.99500 -11.62444   ACIS-S 0.176 (M104)\n
 9532 189.99750 -11.61472   ACIS-I 0.500 (M104)\n


In [ ]:
# Query Heasarc for object in a mission (e.g. M104 with chanmaster)
heasarc = Heasarc()
mission_result = heasarc.query_region(coord, mission, radius='0.2 deg')
if mission_result is not None and len(mission_result) > 0:
    print(f"Found {len(mission_result)} {mission} observations:")
    for obs in mission_result:
        print(f"\nObservation ID: {obs['OBSID']}")
        print(f"https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid={obs['OBSID']}")
else:
    print("No Chandra observations found")

Found 4 chanmaster observations:

Observation ID: 9533
https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid=9533

Observation ID: 9532
https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid=9532

Observation ID: 1586
https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid=1586

Observation ID: 407
https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid=407


In [ ]:
# Downloaded a FITS file and visualize:
def heasarc_image(fits_file):
    with fits.open(fits_file) as hdul:
        data = hdul[0].data
        plt.figure(figsize=(10,10))
        plt.imshow(data, cmap='viridis', origin='lower')
        plt.colorbar()
        plt.show()